# **Identification**

* **Name:** Joaquim Daniel Rios da Cunha
* **Student Number:** 201806651

* **Name:** Pedro Miguel Pinto Silva
* **Student Number:** 201806526

In [1]:
from numpy.matrixlib import mat
import matplotlib.pyplot as plt
from matplotlib.patches import Ellipse
import numpy as np

In [2]:
from google.colab import files
uploaded = files.upload()
# Text file data converted to integer data type
data = np.loadtxt("./dataset.txt", dtype=float)
print(data)

Saving dataset.txt to dataset.txt
[[ 6.         6.         1.5707963 ...  5.2867043  5.1429835  5.1236028]
 [ 5.9696155  6.3472964  1.7453293 ...  4.4675985  4.6891497  4.501786 ]
 [ 5.8793852  6.6840403  1.9198622 ...  4.6946241  4.8221473  4.7143348]
 ...
 [ 5.7320508  5.        13.613568  ...  5.5072914  5.3654677  5.4480188]
 [ 5.8793852  5.3159597 13.788101  ...  5.0121131  5.1567894  4.9275061]
 [ 5.9696155  5.6527036 13.962634  ...  4.6231885  4.7531702  4.8312064]]


In [3]:
robot_x_pos_abs = data[:,0]
robot_y_pos_abs = data[:,1]
robot_ang = data[:,2]

robot_dx = data[:,3]
robot_dy = data[:,4]
robot_d_ang = data[:,5]

lidar_val = data[:,6::]

robot_x_pos_rel = []
robot_y_pos_rel = []
robot_ang_rel = []

robot_x_pos_rel.append(robot_x_pos_abs[0])
robot_y_pos_rel.append(robot_y_pos_abs[0])
robot_ang_rel.append(robot_ang[0])

for i in range(1,len(robot_x_pos_abs),1):

  robot_x_pos_rel.append(robot_x_pos_rel[i-1] + robot_dx[i-1])
  robot_y_pos_rel.append(robot_y_pos_rel[i-1] + robot_dy[i-1])
  robot_ang_rel.append(robot_ang_rel[i-1] + robot_d_ang[i-1])

#Build the cloud points in 2D plan 
map_x_abs = []
map_y_abs = []

map_x_rel = []
map_y_rel = []

for t in range(0,lidar_val.shape[0],1):
  for i in range(lidar_val.shape[1]):    

     map_x_abs.append( robot_x_pos_abs[t] + lidar_val[t][i]*np.cos( ((i-30)*np.pi/180) + robot_ang[t] ) ) 
     map_y_abs.append( robot_y_pos_abs[t] + lidar_val[t][i]*np.sin( ((i-30)*np.pi/180) + robot_ang[t] ) )
     
     map_x_rel.append( robot_x_pos_rel[t] + lidar_val[t][i]*np.cos( ((i-30)*np.pi/180) + robot_ang[t] ) ) 
     map_y_rel.append( robot_y_pos_rel[t] + lidar_val[t][i]*np.sin( ((i-30)*np.pi/180) + robot_ang[t] ) )




# Task 1 - Finding Corners for each measure

In [ ]:
from math import atan
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score

def median_filter(src):
  output = []
  for i in range(0,len(src),1):
    # if it's remaining less then 3 elements, then they will be copy only
    remaining = len(src)-i
    if remaining < 3:
      remaining = -1*remaining
      while remaining < 0:
        output.append(src[len(src)+remaining])
        remaining = remaining + 1
      break
    # Median of 3 elements  
    vec = np.array([src[i],src[i+1],src[i+2]])
    ord_vec = np.sort(vec)
    output.append(ord_vec[1])
  return np.array(output)

def avr_filter(src, size=3):
  output = []
  for i in range(0,len(src),1):
    # if it's remaining less then 3 elements, then they will be copy only
    remaining = len(src)-i
    if remaining < size:
      remaining = -1 * remaining
      while remaining < 0:
        output.append(src[len(src)+remaining])
        remaining = remaining + 1
      break
    # Average of size elements
    sum = 0
    for pos in range(0,size,1):  
      sum = sum + src[i+pos] 
    output.append(sum/size)
  return np.array(output)

def find_max(src):
  max = 0
  idx = 0
  for i in range(0,len(src),1):
    if src[i] > max:
      max = src[i]
      idx = i
  return max,idx

def polar2cart(src):
  ang = np.arange(-30,31,1)
  ang = np.radians(ang)
  x = []
  y = []

  for i in range(len(src)):
    x.append(src[i]*np.cos(ang[i]))
    y.append(src[i]*np.sin(ang[i]))

  return x,y

 
# Function to find the angle between two lines
def find_angle(m1, m2):    
  # Store the tan value  of the angle
  angle = abs((m2 - m1) / (1 + m1 * m2))
  # Calculate tan inverse of the angle and convert to radians
  ret = np.rad2deg(atan(angle)) 
  
  return ret  


corners_row = []
corners_idx = []

has_corner = np.zeros(lidar_val.shape[0])
corner_ang = np.zeros(lidar_val.shape[0])
corner_dist = np.zeros(lidar_val.shape[0])
angd = np.arange(-30,31,1)
for r in range(0,lidar_val.shape[0],1):
  # First we do a filter for noise reduction/atenuation
  #filtered = median_filter(lidar_val[r])
  filtered = avr_filter(lidar_val[r])
  #filtered = lidar_val[r]
  # Look for the maximum distance point and the index, so we can know the angle
  # -30º is for index 0 and 30º for index 60
  max, idx = find_max(filtered)

  # now exists 3 scenarios:
  #   the maximum inside the measures
  #   the maximum in the borders, so the distance inscreases or decreases
  if idx != 0 and idx!=len(filtered)-1:

    # the point isn´t in the border, so could be a corner
    # convert to cartesian coordenates, an take 2 linear aproximations of the points
    x,y = polar2cart(lidar_val[r])
    x_left = x[0:idx]
    y_left = y[0:idx]
    
    # Using sklearn 
    model = linear_model.LinearRegression()
    model.fit(np.array(x_left).reshape(len(x_left),1) , y_left)
    m1 = model.coef_

    x_right = x[idx::]
    y_right = y[idx::]

    model = linear_model.LinearRegression()
    model.fit(np.array(x_right).reshape(len(x_right),1) , y_right)    
    m2 = model.coef_

    ang = find_angle(m1,m2)

    if 0:
      title = "Measure "+ str(r) + " idx: " +str(idx) +" Red: robot pos Green: Max Dist point"
      plt.title(title)
      plt.scatter(x,y)
      plt.scatter(0,0,color='red')
      print(ang)
   
    
    if  ang > 88 and ang < 92:
      corners_row.append(r)
      corners_idx.append(idx)
      
      has_corner[r] = 1
      corner_ang[r] = angd[idx]
      corner_dist[r] = lidar_val[r][idx]
      #plt.scatter(x[idx],y[idx],color='red')

    #plt.grid()
    #plt.show() 

print("Found ",len(corners_row)," corners!")

# Convert corners coordenates to global frame and check if there are multiple
# occurencies for the same corner
corner_pos_glob_x = []
corner_pos_glob_y = []
angd = np.arange(-30,31,1)

for i in range(len(corners_row)):
   row = corners_row[i]
   ang_idx = corners_idx[i]

   x_corner = robot_x_pos_abs[row] + lidar_val[row][ang_idx]*np.cos(np.radians(angd[ang_idx]) + robot_ang[row])
   y_corner = robot_y_pos_abs[row] + lidar_val[row][ang_idx]*np.sin(np.radians(angd[ang_idx]) + robot_ang[row])

   corner_pos_glob_x.append(x_corner)    
   corner_pos_glob_y.append(y_corner)   

plt.figure(figsize=(8,8))
plt.scatter(map_x_abs, map_y_abs, color='b',label="wall")
plt.scatter(corner_pos_glob_x,corner_pos_glob_y,color='red',label="corner")
plt.title('2D Map with absolute position and corners')
plt.xlabel('x (m)')
plt.ylabel('y (m)')
plt.legend(loc='upper left',prop={'size': 12})
plt.grid()
plt.show()

# Task 2 - Position based on robot variations

In [ ]:
plt.figure(figsize=(15,5))
plt.subplot(1,3,1)
plt.title("X evolution")
plt.plot(robot_x_pos_abs, color="blue",label='absolute ')
plt.plot(robot_x_pos_rel, color="green",label='relative')
plt.legend(loc='upper left',prop={'size': 12})
plt.grid()

plt.subplot(1,3,2)
plt.title("Y evolution")
plt.plot(robot_y_pos_abs, color="blue",label='absolute ')
plt.plot(robot_y_pos_rel, color="green",label='relative')
plt.legend(loc='upper left',prop={'size': 12})
plt.grid()

plt.subplot(1,3,3)
plt.title("Theta evolution")
plt.plot(robot_ang, color="blue",label='absolute ')
plt.plot(robot_ang_rel, color="green",label='relative')
plt.legend(loc='upper left',prop={'size': 12})
plt.grid()
plt.show()

plt.figure(figsize=(12,5))
plt.subplot(121)
plt.plot(robot_x_pos_abs,robot_y_pos_abs,color="blue", label='absolute ')
plt.plot(robot_x_pos_rel,robot_y_pos_rel,color="green", label='relative')
plt.title("Robot Location ")
plt.legend(loc='upper left',prop={'size': 12})
plt.grid()

plt.subplot(122)
plt.scatter(map_x_abs, map_y_abs, color='blue',s = 0.5, label='absolute' )
plt.scatter(map_x_rel, map_y_rel, color='green',s = 0.5, label='relative')
plt.title("2D Map")
plt.legend(loc='upper left',prop={'size': 12})
plt.grid()
plt.xlabel('x (m)')
plt.ylabel('y (m)')
plt.show()



In [ ]:
#incremento a cada posiçao para a demonstraçao

for i in range( 0, robot_x_pos_abs.shape[0], 1):

  plt.figure(figsize=(7,7))
  plt.plot(robot_x_pos_abs,robot_y_pos_abs,color="blue", label='absolute ')
  plt.scatter(robot_x_pos_rel[0:i],robot_y_pos_rel[0:i],color="green", label='relative')
  plt.title("Robot Location ")
  plt.legend(loc='upper left',prop={'size': 12})
  plt.grid()
  plt.show()

# Task 3 - EKF for robot and corners positions


In [ ]:
import scipy.stats as stats
import math

# Estimate variances of X, Y, theta

# error in x
err_x = (robot_x_pos_abs - robot_x_pos_rel)**2
# Fit a normal distribution to the data:
mu_x, std_x = stats.norm.fit(err_x)

plt.figure(figsize=(5,5))
plt.title("Squared Error X ")
x = np.linspace(mu_x - 3*std_x, mu_x + 3*std_x, 100)
plt.plot(x, stats.norm.pdf(x, mu_x, std_x))
plt.hist(err_x,bins=len(err_x),rwidth=0.9)
plt.grid()

print("X: mean ", mu_x , " deviation", std_x)

# error in y
err_y = (robot_y_pos_abs - robot_y_pos_rel)**2
# Fit a normal distribution to the data:
mu_y, std_y = stats.norm.fit(err_y)

plt.figure(figsize=(5,5))
plt.title("Squared Error Y")
y = np.linspace(mu_y - 3*std_y, mu_y + 3*std_y, 100)
plt.plot(y, stats.norm.pdf(y, mu_y, std_y))
plt.hist(err_y,bins=len(err_y),rwidth=0.9)
plt.grid()
print("Y: ", mu_y , " ", std_y)

# error in theta
err_theta = (robot_ang - robot_ang_rel)**2
# Fit a normal distribution to the data:
mu_theta, std_theta = stats.norm.fit(err_theta)

plt.figure(figsize=(5,5))
plt.title("Squared Error theta")
theta = np.linspace(mu_theta - 3*std_theta, mu_theta + 3*std_theta, 100)
plt.plot(theta, stats.norm.pdf(theta, mu_theta, std_theta))
plt.hist(err_theta,bins=len(err_theta),rwidth=0.9)
plt.grid()

print("theta: ", mu_theta , " ", std_theta)

def dist(X1, X2):
  return np.sqrt( (X1[0]-X2[0])**2+(X1[1]-X2[1])**2 )


c1 = []
c2 = []
c3 = []
c4 = []
c5 = []

for i in range(len(corner_pos_glob_x)):
  
  X1 = ([corner_pos_glob_x[i], corner_pos_glob_y[i]])

  if len(c1) == 0:
    c1.append(X1)
    continue
  else:
    if dist(X1,c1[0]) < 2:
      c1.append(X1)
      continue

  if len(c2) == 0:
    c2.append(X1)
    continue
  else:
    if dist(X1,c2[0]) < 2:
      c2.append(X1)
      continue

  if len(c3) == 0:
    c3.append(X1)
    continue
  else:
    if dist(X1,c3[0]) < 2:
      c3.append(X1)
      continue

  if len(c4) == 0:
    c4.append(X1)
    continue
  else:
    if dist(X1,c4[0]) < 2:
      c4.append(X1)
      continue

  if len(c5) == 0:
    c5.append(X1)
    continue
  else:
    if dist(X1,c5[0]) < 2:
      c5.append(X1)
      continue

mean = []
c = [c1,c2,c3,c4,c5]

for i in range(0,5):
  sum_x = 0
  sum_y = 0
  for j in range(len(c[i])):
    sum_x = sum_x + c[i][j][0] 
    sum_y = sum_y + c[i][j][1]
  mean.append((sum_x/len(c[i]) , sum_y/len(c[i])))

error_x = []
error_y = []
for i in range(0,5):
  for j in range(len(c[i])):
    error_x.append( (mean[i][0] - c[i][j][0])**2 )
    error_y.append( (mean[i][1] - c[i][j][1])**2 )

# Fit a normal distribution to the data:
mu_xc, std_xc = stats.norm.fit(error_x)

print("Xc: ", mu_xc , " ", std_xc)

# Fit a normal distribution to the data:
mu_yc, std_yc = stats.norm.fit(error_y)

print("Yc: ", mu_yc , " ", std_yc)


In [7]:
import statistics as st

variance_x_abs = st.variance(robot_x_pos_abs)
print("Var x abs: ",variance_x_abs)

variance_x_rel = st.variance(robot_x_pos_rel)
print("Var x rel: ",variance_x_rel)

var_x = st.variance(robot_x_pos_abs - robot_x_pos_rel)
print("Var x_abs - x_rel : ",var_x)
print("")

variance_y_abs = st.variance(robot_y_pos_abs)
print("Var y abs: ",variance_y_abs)

variance_y_rel = st.variance(robot_y_pos_rel)
print("Var y rel: ",variance_y_rel)

var_y = st.variance(robot_y_pos_abs - robot_y_pos_rel)
print("Var y_abs - y_rel : ",var_y)
print("")
variance_theta_abs = st.variance(robot_ang)
print("Var theta abs: ",variance_theta_abs)

variance_theta_rel = st.variance(robot_ang_rel)
print("Var theta rel: ",variance_theta_rel)

var_theta = st.variance(robot_ang - robot_ang_rel)
print("Var theta_abs - theta_rel : ",var_theta)

Var x abs:  2.0281689955419977
Var x rel:  2.2194235588862914
Var x_abs - x_rel :  0.21287745700578892

Var y abs:  2.0281689955419973
Var y rel:  2.087727279957471
Var y_abs - y_rel :  0.016089523828419366

Var theta abs:  13.342242874062173
Var theta rel:  11.704975053883906
Var theta_abs - theta_rel :  0.09625299714601422


In [8]:
import matplotlib.pyplot as plt
import numpy as np
from numpy import dot 
from numpy import * 
from numpy.linalg import inv 
from numpy.linalg import det


def get_H(Xr, Xc, pos, n): 

  # H
  da_dxr = (Xc[1] - Xr[1]) / ( (Xc[0] - Xr[0])**2 + (Xc[1] - Xr[1])**2 ) 
  da_dyr = (-Xc[0] + Xr[0]) / ( (Xc[0] - Xr[0])**2 + (Xc[1] - Xr[1])**2 )
  da_dor = -1

  dR_dxr = (-Xc[0] + Xr[0]) / np.sqrt( (-Xc[0] + Xr[0])**2 + (Xc[1] - Xr[1])**2 )
  dR_dyr = (-Xc[1] + Xr[1]) / np.sqrt( (-Xc[0] + Xr[0])**2 + (Xc[1] - Xr[1])**2 )
  dR_dor = 0 

  H = np.array([[ da_dxr.item(0), da_dyr.item(0), da_dor ] , [ dR_dxr.item(0), dR_dyr.item(0), dR_dor ] ])   

  # Expand H - increase with 0 
  new_shape = (H.shape[0], H.shape[1] + 2*n)
  shape_diff = np.array(new_shape) - np.array(H.shape)
  H = np.lib.pad(H, ((0,shape_diff[0]),(0,shape_diff[1])), 'constant')  

  Z_pred = np.array([[0.0],[0.0]], dtype = float)
  
  if pos > 0 :
    # Put the correct values of feature pos - with the correct values
    da_dxc = (Xr[1] - Xc[1]) / ( (Xc[0] - Xr[0])**2 + (Xc[1] - Xr[1])**2 )
    da_dyc = (-Xr[0] + Xc[0]) / ( (Xc[0] - Xr[0])**2 + (Xc[1] - Xr[1])**2 )

    dR_dxc = (Xc[0] - Xr[0]) / np.sqrt( (-Xc[0] + Xr[0])**2 + (Xc[1] - Xr[1])**2 )
    dR_dyc = (Xc[1] - Xr[1]) / np.sqrt( (-Xc[0] + Xr[0])**2 + (Xc[1] - Xr[1])**2 )

    # For alpha
    H[0,pos] = da_dxc
    H[0,pos+1] = da_dyc

    # For distance R
    H[1,pos] = dR_dxc
    H[1,pos+1] = dR_dyc

    Z_pred[0] = math.atan2(Xr[pos+1]-Xr[1],Xr[pos]-Xr[0]) - Xr[2]
    Z_pred[1] = np.sqrt( (Xr[pos+1]-Xr[1])**2 + (Xr[pos]-Xr[0])**2)

  return (H,Z_pred)

def incr_vec(X_in,incr_size):
  
  X_in = np.resize(X_in, (X_in.shape[0] + incr_size, 1) )

  return X_in

def incr_mtx(X_in, incr_size):

  new_shape = (X_in.shape[0] + incr_size, X_in.shape[1] + incr_size)
  shape_diff = np.array(new_shape) - np.array(X_in.shape)
  X_in = np.lib.pad(X_in, ((0,shape_diff[0]),(0,shape_diff[1])), 'constant')   

  return X_in

def euc_distance(x1, y1, x2, y2):
  return np.sqrt( (x1-x2)**2+(y1-y2)**2 )

def is_new(Z_meas, X_k_next):
  shape = X_k_next.shape[0]
  if shape <= 3: 
    return (True,0)

  for i in range(3,shape,2):
    if( euc_distance( Z_meas[0], Z_meas[1], X_k_next[i], X_k_next[i+1]) < 4 ):
      return (False,i)
  return (True,0)  


def kf_predict(X, P, A, Q, F, U): 
  X = A @ X + F @ U 
  P = A @ P @ A.T + Q
  return(X,P) 

def kf_update(X, P, Y, H, R, Z_pred): 
  IS = H @ P @ H.T + R 
  K = P @ H.T @ inv(IS)
  #HX = H@X  no EKF esta linearização não é valida!
  #diff = Y - HX
  diff = Y - Z_pred
  while(diff[0] > np.pi):
    diff[0] = diff[0] - 2*np.pi
  while(diff[0] < -np.pi):
    diff[0] = diff[0] + 2*np.pi

  aux = K @ diff 
  X = X + aux
  P = P - K @ IS @ K.T 
    
  return (X,P) 

In [9]:
def draw_ellipses(X,I,rot_ang=0):
  u = X[0]          #x-position of the center
  v = X[1]          #y-position of the center
  a = np.sqrt(I[0]) * 0.5    #radius on the x-axis
  b = np.sqrt(I[1]) * 0.5    #radius on the y-axis
  t_rot = rot_ang   #rotation angle

  t = np.linspace(0, 2*np.pi, 100)
  Ell = np.array([a*np.cos(t) , b*np.sin(t)])  
      #u,v removed to keep the same center location
  R_rot = np.array([[cos(t_rot) , -sin(t_rot)],[sin(t_rot) , cos(t_rot)]])  
      #2-D rotation matrix

  Ell_rot = np.zeros((2,Ell.shape[1]))
  for i in range(Ell.shape[1]):
      Ell_rot[:,i] = np.dot(R_rot,Ell[:,i])

  return (u + Ell_rot[0,:], v + Ell_rot[1,:])

In [ ]:
#history of robot positions (for printing)
x_pos=[]
y_pos=[]
theta_pos=[]

# x y theta
X_k = np.array( [ [robot_x_pos_abs[0]] , [robot_y_pos_abs[0]] , [robot_ang[0]] ], dtype=float  )

# dx dy dtheta
U_k = np.array( [ [0.0] , [0.0] , [0.0] ], dtype=float  )

X_k_next = X_k


A = np.array( [ [ 1.0, 0.0, 0.0 ] ,
                [ 0.0, 1.0, 0.0 ] ,
                [ 0.0, 0.0, 1.0 ] ] )

F = np.array( [ [ 1.0, 0.0, 0.0 ] ,
                [ 0.0, 1.0, 0.0 ] ,
                [ 0.0, 0.0, 1.0 ] ] )

P_k = np.array( [ [ 0.1, 0.0, 0.0 ] ,
                  [ 0.0, 0.1, 0.0 ] ,
                  [ 0.0, 0.0, 0.1 ] ],  dtype=float )

P_k_next = P_k

#valores extraídos a partir dos resultados das tasks anteriores
#Q = np.array( [ [ std_x**2, 0.0, 0.0 ] ,
#                [ 0.0, std_y**2, 0.0 ] ,
#                [ 0.0, 0.0, std_theta**2 ] ] )

Q = np.array( [ [ var_x, 0.0, 0.0 ] ,
                [ 0.0, var_y, 0.0 ] ,
                [ 0.0, 0.0, var_theta ] ] )

# tentar obter isto de algum lado
R = np.array([ [ 0.01, 0.0 ] ,
               [ 0.0, 0.05 ] ] )

n_corner_detected = 0
corner_orientation = []
for t in range(0,lidar_val.shape[0],1):
 
  U_k[0] = robot_dx[t]
  U_k[1] = robot_dy[t]
  U_k[2] = robot_d_ang[t]
  (X_k_next, P_k_next) = kf_predict(X_k, P_k, A, Q, F, U_k)

  if has_corner[t] == 1:
    
    x_corner = X_k_next[0] + corner_dist[t] * np.cos( X_k_next[2] + np.radians(corner_ang[t]) )
    y_corner = X_k_next[1] + corner_dist[t] * np.sin( X_k_next[2] + np.radians(corner_ang[t]) )
    
    XY_corner_pred = np.array([ x_corner , y_corner ])
    Z_meas_aR = np.array([ [np.radians(corner_ang[t])] , [corner_dist[t]] ]) 
    
    # Is new or similar to an existing feature
    (not_found, f_idx) = is_new(XY_corner_pred, X_k_next)

    if not_found: #new
      # Add new feature - increase state X and covarianve P    
      # For state X
      X_k_next = incr_vec(X_k_next, 2)
      # For covariance P
      P_k_next = incr_mtx(P_k_next, 2) 
      
      # update number of features
      n_corner_detected = n_corner_detected + 1
      # save orientation of the robot when detected corner - only for graphical purposes
      corner_orientation.append(X_k_next[2,0])

      X_k_next[2+2*n_corner_detected-1] = x_corner
      X_k_next[2+2*n_corner_detected] = y_corner

      P_k_next[2+2*n_corner_detected-1 , 2+2*n_corner_detected-1] = P_k_next[0,0] 
      P_k_next[2+2*n_corner_detected   , 2+2*n_corner_detected  ] = P_k_next[1,1] 
      
      # Extend other matrices A, F, Q e U

      A = incr_mtx(A, 2)
      A[2+2*n_corner_detected-1 , 2+2*n_corner_detected-1] = 1
      A[2+2*n_corner_detected   , 2+2*n_corner_detected  ] = 1
      
      F = incr_mtx(F, 2)
      F[2+2*n_corner_detected-1 , 2+2*n_corner_detected-1] = 1
      F[2+2*n_corner_detected   , 2+2*n_corner_detected  ] = 1
      
      Q = incr_mtx(Q, 2)
      Q[2+2*n_corner_detected-1 , 2+2*n_corner_detected-1] = 0 # deve ser 0 para nao mudar a variancia do canto no predict
      Q[2+2*n_corner_detected   , 2+2*n_corner_detected  ] = 0 

      U_k = incr_vec(U_k, 2)
      U_k[2+2*n_corner_detected-1] = 0
      U_k[2+2*n_corner_detected] = 0

    else: # is identical to an existing  
      # f_idx has feature index
      (H, Z_pred) = get_H(X_k_next, XY_corner_pred, f_idx, n_corner_detected)
      (X_k_next, P_k_next) = kf_update(X_k_next, P_k_next, Z_meas_aR, H, R, Z_pred) 
      corner_orientation[f_idx%2 - 1] = X_k_next[2,0]

  X_k = X_k_next
  P_k = P_k_next

  x_pos.append(X_k[0])
  y_pos.append(X_k[1])
  theta_pos.append(X_k[2])


  if 1:
    plt.figure(figsize=(10,10))

    plt.scatter(map_x_abs, map_y_abs, color='blue', label='2D Map')
    plt.scatter(corner_pos_glob_x,corner_pos_glob_y,color='red', label='corners from Task1')
    
    plt.plot(robot_x_pos_abs,robot_y_pos_abs,color="blue", label='robot abs trajectory ')

    # Plot robot and uncertainty ellipses
    plt.scatter(X_k_next[0],X_k_next[1],color="green", label='robot pos') 
    (x_pos_ell,y_pos_ell) = draw_ellipses((X_k_next[0,0],X_k_next[1,0]), (P_k_next[0,0],P_k_next[1,1]),X_k_next[2,0])
    plt.plot(x_pos_ell, y_pos_ell, color="green")

    # Plot corners estimation and uncertainty ellipses
    for i in range(3,3+2*n_corner_detected,2):        
      (xc_pos_ell,yc_pos_ell) = draw_ellipses( (X_k_next[i,0],X_k_next[i+1,0]), (P_k_next[i,i],P_k_next[i+1,i+1]),corner_orientation[i%2 - 1] ) 
      plt.scatter(X_k_next[i,0],X_k_next[i+1,0],color='orange') 
      plt.plot(xc_pos_ell, yc_pos_ell, color='orange')#, label='corner estimation')
               
    
    # Old positions of robot 
    plt.scatter(x_pos,y_pos,color='green')
    plt.grid()
    plt.legend(loc='upper left')
    plt.show()



t_axis = np.linspace(0,len(x_pos),len(x_pos),endpoint=False)

plt.figure(figsize=(20,5))
plt.subplot(1,3,1)
plt.title("X evolution")
plt.plot(robot_x_pos_abs, color="blue",label='absolute')
plt.scatter(t_axis, x_pos, c="green", label='estimation')
plt.grid()

plt.subplot(1,3,2)
plt.title("Y evolution")
plt.plot(robot_y_pos_abs, color="blue",label='absolute')
plt.scatter(t_axis,y_pos, c="green", label='estimation')
plt.grid()

plt.subplot(1,3,3)
plt.title("Theta evolution")
plt.plot(robot_ang, color="blue", label='absolute')
plt.scatter(t_axis,theta_pos, c="green", label='estimation')
plt.grid()

plt.show()

